In [1]:
#default_exp rescore.percolator

In [3]:
#export
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
import numba
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch

from alphabase.peptide.fragment import get_charged_frag_types

from alphadeep.rescore.feature_extractor import (
    ScoreFeatureExtractor
)

In [61]:
#export
@numba.njit
def fdr_to_q_values(fdr_values):
    q_values = np.zeros_like(fdr_values)
    min_q_value = np.max(fdr_values)
    for i in range(len(fdr_values) - 1, -1, -1):
        fdr = fdr_values[i]
        if fdr < min_q_value:
            min_q_value = fdr
        q_values[i] = min_q_value
    return q_values

def get_q_values(_df, score_column, decoy_column='decoy'):
    _df = _df.reset_index()
    _df = _df.sort_values([score_column,score_column], ascending=False)
    target_values = 1-_df[decoy_column].values
    decoy_cumsum = np.cumsum(_df[decoy_column].values)
    target_cumsum = np.cumsum(target_values)
    fdr_values = decoy_cumsum/target_cumsum
    _df['fdr'] = fdr_to_q_values(fdr_values)
    return _df

@numba.njit
def score_to_q_value(scores, base_scores, base_q_values):
    q_values = np.zeros_like(scores)
    i,j = 0,0
    while i < len(scores) and j < len(base_scores):
        if scores[i] >= base_scores[j]:
            q_values[i] = base_q_values[j]
            i += 1
        else:
            j += 1
    while i < len(scores):
        q_values[i] = base_q_values[-1]
        i += 1
    return q_values

class DeepLearningScore(torch.nn.Module):
    def __init__(self, in_features, nlayer=8):
        super().__init__()
        hidden = 128
        self.nn = torch.nn.Sequential(
            torch.nn.Linear(in_features, hidden),
            torch.nn.ReLU(),
            *[]
        )

class Percolator:
    def __init__(self,
        ml_type='logistic_regression', #or 'random_forest'
        cv_fold = 2,
        n_iteration = 5,
        ms2_ppm = True, ms2_tol=30,
        **sklearn_kwargs
    ):
        self.charged_frag_types = get_charged_frag_types(
            ['b','y'], 2
        )
        self.ms2_ppm = ms2_ppm
        self.ms2_tol = ms2_tol
        if ml_type == 'logistic_regression':
            ml_type = 'lr'
        self.ml_type = ml_type
        self.fdr_level = 'psm'
        self.fdr = 0.01
        self.cv_fold = cv_fold
        self.n_iter = n_iteration

        if ml_type == 'lr':
            self.model = LogisticRegression(
                solver='liblinear', **sklearn_kwargs
            )
        else:
            self.model = RandomForestClassifier(**sklearn_kwargs)
        self.feature_extractor = ScoreFeatureExtractor(
            ppm=ms2_ppm, tol=ms2_tol
        )
        self.feature_list = self.feature_extractor.score_feature_list
        self.feature_list += ['score','nAA','charge']
        #self.feature_list.append('ml_score')

    def enable_model_fine_tuning(self):
        self.feature_extractor.model_fine_tuning = True
    def disable_model_fine_tuning(self):
        self.feature_extractor.model_fine_tuning = False

    def _estimate_fdr(self, df):
        df = df.sort_values(['ml_score','decoy'], ascending=False)
        df = df.reset_index(drop=True)
        if self.fdr_level == 'psm':
            target_values = 1-df['decoy'].values
            decoy_cumsum = np.cumsum(df['decoy'].values)
            target_cumsum = np.cumsum(target_values)
            fdr_values = decoy_cumsum/target_cumsum
            df['fdr'] = fdr_to_q_values(fdr_values)
        else:
            if self.fdr_level == 'precursor':
                _df = df.groupby([
                    'sequence','mods','mod_sites','charge','decoy'
                ])['ml_score'].max()
            elif self.fdr_level == 'peptide':
                _df = df.groupby([
                    'sequence','mods','mod_sites','decoy'
                ])['ml_score'].max()
            else:
                _df = df.groupby(['sequence','decoy'])['ml_score'].max()
            _df = _df.reset_index()
            _df = _df.sort_values(['ml_score','decoy'], ascending=False)
            target_values = 1-_df['decoy'].values
            decoy_cumsum = np.cumsum(_df['decoy'].values)
            target_cumsum = np.cumsum(target_values)
            fdr_values = decoy_cumsum/target_cumsum
            _df['fdr'] = fdr_to_q_values(fdr_values)
            df['fdr'] = score_to_q_value(
                df['ml_score'].values, _df['ml_score'].values, 
                _df['fdr'].values
            )
        return df

    def _cv_score(self, df):
        df = df.sample(frac=1).reset_index(drop=True)
        df_target = df[df.decoy == 0]
        df_decoy = df[df.decoy != 0]
        
        if self.cv_fold > 1:
            test_df_list = []
            for i in range(self.cv_fold):
                t_mask = np.ones(len(df_target), dtype=bool)
                _slice = slice(i, len(df_target), self.cv_fold)
                t_mask[_slice] = False
                cv_df_target = df_target[t_mask]
                train_t_df = cv_df_target[
                    cv_df_target.fdr <= self.fdr
                ]
                test_t_df = df_target[_slice]
                
                d_mask = np.ones(len(df_decoy), dtype=bool)
                _slice = slice(i, len(df_decoy), self.cv_fold)
                d_mask[_slice] = False
                train_d_df = df_decoy[d_mask]
                test_d_df = df_decoy[_slice]

                train_df = pd.concat((train_t_df, train_d_df))
                train_label = np.ones(len(train_df),dtype=np.int32)
                train_label[len(train_t_df):] = 0
                test_df = pd.concat((test_t_df, test_d_df))

                self.model.fit(
                    train_df[
                        self.feature_list
                    ].values, train_label
                )
                if self.ml_type == 'lr':
                    test_df['ml_score'] = self.model.decision_function(
                        test_df[self.feature_list].values
                    )
                else:
                    test_df['ml_score'] = self.model.predict_proba(
                        test_df[self.feature_list].values
                    )[:,1]
                test_df_list.append(test_df)
        
            return pd.concat(test_df_list)
        else:
            train_t_df = df_target[df_target.fdr <= self.fdr]
            train_df = pd.concat((train_t_df, df_decoy))
            train_label = np.ones(len(train_df),dtype=np.int32)
            train_label[len(train_t_df):] = 0
            test_df = pd.concat((df_target, df_decoy))

            self.model.fit(train_df[self.feature_list].values, train_label)
            if self.ml_type == 'lr':
                test_df['ml_score'] = self.model.decision_function(
                    test_df[self.feature_list].values
                )
            else:
                test_df['ml_score'] = self.model.predict_proba(
                    test_df[self.feature_list].values
                )[:,1]
        
            return test_df

    def extract_features(self,
        psm_df, ms2_file_dict:dict, ms2_file_type:str
    ):
        psm_df['ml_score'] = psm_df.score
        psm_df = self._estimate_fdr(psm_df)
        print('Extracting features ...')
        psm_df = self.feature_extractor.extract_features(
            psm_df, ms2_file_dict, 
            ms2_file_type, 
            psm_tune_df=psm_df[(psm_df.fdr<0.01)&(psm_df.decoy==0)],
            frag_types_to_match=self.charged_frag_types, 
            ms2_ppm=self.ms2_ppm, ms2_tol=self.ms2_tol
        )
        print('End extracting features ...')
        return psm_df

    def re_score(self, df):
        print(f'{len(df[(df.fdr<=self.fdr) & (df.decoy==0)])} target PSMs at {self.fdr} psm-level FDR')
        for i in range(self.n_iter):
            print(f'[RUN] Iteration {i+1} of Percolator ...')
            df = self._cv_score(df)
            df = self._estimate_fdr(df)
            print(f'[RUN] {len(df[(df.fdr<=self.fdr) & (df.decoy==0)])} target PSMs at {self.fdr} psm-level FDR')
        df = self._estimate_fdr(df)
        print(f'[END] {len(df[(df.fdr<=self.fdr) & (df.decoy==0)])} target PSMs at {self.fdr} {self.fdr_level}-level FDR')
        return df

    def run(self,
        psm_df, ms2_file_dict:dict, ms2_file_type:str
    ):
        df = self.extract_features(
            psm_df, ms2_file_dict, ms2_file_type
        )
        return self.re_score(df)